# General description of the data

In this document is gonna be the process for do the data scraping to tyhe sites of industrial assets in belgium. 


DataFrame (url_fiche)
        ↓
Scrape PDF links from each page
        ↓
Download PDFs
        ↓
Extract text from PDFs
        ↓
Translate NL → EN
        ↓
LLM (Ollama) extraction & evaluation
        ↓
Structured assets dataset (JSON / DataFrame)


## Install the needed libraries

In [1]:
!pip install pdfplumber pandas tqdm argostranslate langdetect
!pip install geopandas
!pip install fiona
!pip install cloudscraper
!pip install ollama

## Import the libraries

In [2]:
import geopandas as gpd
import fiona
import pandas as pd
import ollama
from ollama import Client
import requests
from bs4 import BeautifulSoup
import argostranslate.package
import argostranslate.translate
import requests
import json
from tqdm import tqdm
import pandas as pd
import os
from concurrent.futures import ProcessPoolExecutor, as_completed
from worker_utils import process_document_task

Note: Dowloading the Language needs for this project !

In [3]:
argostranslate.package.update_package_index()

available_packages = argostranslate.package.get_available_packages()

languages_needed = {
    ("fr", "en"),
    ("nl", "en"),
    ("de", "en"),
}

for pkg in available_packages:
    if (pkg.from_code, pkg.to_code) in languages_needed:
        argostranslate.package.install_from_path(pkg.download())


## Treatment of the initial data found

In this part i did a treatment for have the data in a good datafrae readable way ! 

In [ ]:
# Define the file name
gpkg_file = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Scraping-PDFs-LLM\data\raw\pf_gpbv.gpkg"

## --- Step 1: List all available layers (datasets) ---
# A GeoPackage often contains multiple datasets (layers).
# We use fiona (the library geopandas relies on) to list them.

try:
    available_layers = fiona.listlayers(gpkg_file)
    print(f"File: {gpkg_file}")
    print(f"Available layers: {available_layers}")
except fiona.errors.DriverError as e:
    # If the file is not found or is corrupted
    print(f"Error listing layers: {e}")

# If layers were found, you can proceed to read one
if available_layers:
    # --- Step 2: Read a specific layer into a GeoDataFrame ---
    
    # Let's assume you want to read the first layer found
    layer_to_read = available_layers[0]

    print(f"\nReading layer: '{layer_to_read}'...")
    
    # Read the data into a GeoDataFrame (gdf)
    gdf = gpd.read_file(gpkg_file, layer=layer_to_read)

    # Display the first few rows and the data type
    print("\nSuccessfully loaded data:")
    print(gdf.head())
    print(f"\nData type: {type(gdf)}")
    print(f"Coordinate Reference System (CRS): {gdf.crs}")

else:
    print("\nCould not find any layers to read.")

File: C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Scraping-PDFs-assets\data\raw\pf_gpbv.gpkg
Available layers: ['pf_gpbv', 'layer_styles']

Reading layer: 'pf_gpbv'...

Successfully loaded data:
   id                        nummer  \
0   1  BE.VL.000000002.INSTALLATION   
1   2  BE.VL.000000004.INSTALLATION   
2   3  BE.VL.000000006.INSTALLATION   
3   4  BE.VL.000000007.INSTALLATION   
4   5  BE.VL.000000008.INSTALLATION   

                                                naam       type   gemeente  \
0                               AGC Glass Europe Mol  industrie        Mol   
1                             Agfa Gevaert_Gevaert 1  industrie    Mortsel   
2                              Agfa Gevaert_Gevaert5  industrie    Mortsel   
3  Air Liquide Large Industry_Syngas (blokveld E6...  industrie  Antwerpen   
4  Air Liquide Large Industry_Jupiter (blokveld G...  industrie  Antwerpen   

  postcode   straatnaam huisnummer busnummer vlarem_rubriek_num

c:\Users\m.saenzortiz\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on 'C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Scraping-PDFs-assets\data\raw\pf_gpbv.gpkg'
  return ogr_read(


## Import the DataSet ! 


Note: This extract is also after a data treatment for segmentation

In [8]:
gdf= pd.read_excel("gpbv_data_segment_subseg.xlsx")

gdf

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,Main,Sub,Category,Sub-Category,rie_rubriek_nummer,vlarem_rubriek_omschrijving,url_fiche,geometry
0,1,BE.VL.000000002.INSTALLATION,AGC Glass Europe Mol,industrie,Mol,2400,Voortstraat,27,NaN,20.3.4.1°b),20,3,Industrial installations,Non-metallic mineral products industry,3.3.,20.3.4.1°b) - meer dan 20 ton,https://gpbv.omgeving.vlaanderen.be/gpbv-regis...,POINT (205620 209625)
1,2,BE.VL.000000004.INSTALLATION,Agfa Gevaert_Gevaert 1,industrie,Mortsel,2640,Septestraat,27,NaN,7.11.2°d),7,11,Chemical industry,Manufacture of chemical substances (industrial...,4.2.d),7.11.2°d) - van zouten,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (155100.25 207532.25)
2,3,BE.VL.000000006.INSTALLATION,Agfa Gevaert_Gevaert5,industrie,Mortsel,2640,Septestraat,27,NaN,7.11.1°h),7,11,Chemical industry,Manufacture of chemical substances (industrial...,4.1.h),7.11.1°h) - kunststofmaterialen,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (155188 207854)
3,4,BE.VL.000000007.INSTALLATION,Air Liquide Large Industry_Syngas (blokveld E6...,industrie,Antwerpen,2040,Scheldelaan,600,NaN,7.11.2°a),7,11,Chemical industry,Manufacture of chemical substances (industrial...,4.2.a),7.11.2°a) - van gassen,https://gpbv.omgeving.vlaanderen.be/installati...,POINT (142907.25 228088.5)
4,5,BE.VL.000000008.INSTALLATION,Air Liquide Large Industry_Jupiter (blokveld G...,industrie,Antwerpen,2040,Scheldelaan,600,NaN,7.11.2°a),7,11,Chemical industry,Manufacture of chemical substances (industrial...,4.2.a),7.11.2°a) - van gassen,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (142738 228494)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,1956,BE.VL.000002398.INSTALLATION,Verhee Luc,veeteelt,Heuvelland,8950,Nieuwkerkestraat(Nieuwk),86,NaN,9.3.1.d),9,3,Animal husbandry,Poultry,6.6.a),9.3.1.d) - intensieve pluimveehouderij met mee...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (42426.773 161415.77300000002)
1956,1957,BE.VL.000002399.INSTALLATION,BoVaKip,veeteelt,Zulte,9870,Ponteputstraat,4,NaN,9.3.1.d),9,3,Animal husbandry,Poultry,6.6.a),9.3.1.d) - intensieve pluimveehouderij met mee...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (87716.95 180753.30000000002)
1957,1958,BE.VL.000002400.INSTALLATION,Vanexem Geert,veeteelt,Vleteren,8640,Koekuitstraat(West),4,NaN,9.5.d)1°,9,5,Animal husbandry,Mixed animal facility,6.6.a),9.5.d)1° - een intensieve pluimvee- of varkens...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (35057.3 177059.4)
1958,1959,BE.VL.000002401.INSTALLATION,Dekervel Bart en Dekervel Aaron,veeteelt,Poperinge,8972,Couthoflaan,13,NaN,9.3.1.d),9,3,Animal husbandry,Poultry,6.6.a),9.3.1.d) - intensieve pluimveehouderij met mee...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (29486 175733)


## Geting tha PDFs url from the differents sites

In [9]:
def get_pdf_links(url):
    r = requests.get(url, timeout=15)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    pdf_links = []

    for a in soup.select("div.jumbotron a[href]"):
        link_text = a.get_text(strip=True).lower()
        href = a["href"]

        if link_text.endswith(".pdf"):
            pdf_links.append(href)

    return list(set(pdf_links)) 


In [10]:
gdf["pdf_links"] = gdf["url_fiche"].apply(get_pdf_links)
gdf = gdf.explode("pdf_links")
gdf

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,Main,Sub,Category,Sub-Category,rie_rubriek_nummer,vlarem_rubriek_omschrijving,url_fiche,geometry,pdf_links
0,1,BE.VL.000000002.INSTALLATION,AGC Glass Europe Mol,industrie,Mol,2400,Voortstraat,27,NaN,20.3.4.1°b),20,3,Industrial installations,Non-metallic mineral products industry,3.3.,20.3.4.1°b) - meer dan 20 ton,https://gpbv.omgeving.vlaanderen.be/gpbv-regis...,POINT (205620 209625),https://gpbv.omgeving.vlaanderen.be/gpbv-regis...
0,1,BE.VL.000000002.INSTALLATION,AGC Glass Europe Mol,industrie,Mol,2400,Voortstraat,27,NaN,20.3.4.1°b),20,3,Industrial installations,Non-metallic mineral products industry,3.3.,20.3.4.1°b) - meer dan 20 ton,https://gpbv.omgeving.vlaanderen.be/gpbv-regis...,POINT (205620 209625),https://gpbv.omgeving.vlaanderen.be/gpbv-regis...
0,1,BE.VL.000000002.INSTALLATION,AGC Glass Europe Mol,industrie,Mol,2400,Voortstraat,27,NaN,20.3.4.1°b),20,3,Industrial installations,Non-metallic mineral products industry,3.3.,20.3.4.1°b) - meer dan 20 ton,https://gpbv.omgeving.vlaanderen.be/gpbv-regis...,POINT (205620 209625),https://gpbv.omgeving.vlaanderen.be/gpbv-regis...
0,1,BE.VL.000000002.INSTALLATION,AGC Glass Europe Mol,industrie,Mol,2400,Voortstraat,27,NaN,20.3.4.1°b),20,3,Industrial installations,Non-metallic mineral products industry,3.3.,20.3.4.1°b) - meer dan 20 ton,https://gpbv.omgeving.vlaanderen.be/gpbv-regis...,POINT (205620 209625),https://gpbv.omgeving.vlaanderen.be/gpbv-regis...
0,1,BE.VL.000000002.INSTALLATION,AGC Glass Europe Mol,industrie,Mol,2400,Voortstraat,27,NaN,20.3.4.1°b),20,3,Industrial installations,Non-metallic mineral products industry,3.3.,20.3.4.1°b) - meer dan 20 ton,https://gpbv.omgeving.vlaanderen.be/gpbv-regis...,POINT (205620 209625),https://gpbv.omgeving.vlaanderen.be/gpbv-regis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,1956,BE.VL.000002398.INSTALLATION,Verhee Luc,veeteelt,Heuvelland,8950,Nieuwkerkestraat(Nieuwk),86,NaN,9.3.1.d),9,3,Animal husbandry,Poultry,6.6.a),9.3.1.d) - intensieve pluimveehouderij met mee...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (42426.773 161415.77300000002),https://gpbv.omgeving.vlaanderen.be//api/view/...
1956,1957,BE.VL.000002399.INSTALLATION,BoVaKip,veeteelt,Zulte,9870,Ponteputstraat,4,NaN,9.3.1.d),9,3,Animal husbandry,Poultry,6.6.a),9.3.1.d) - intensieve pluimveehouderij met mee...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (87716.95 180753.30000000002),https://gpbv.omgeving.vlaanderen.be//api/view/...
1957,1958,BE.VL.000002400.INSTALLATION,Vanexem Geert,veeteelt,Vleteren,8640,Koekuitstraat(West),4,NaN,9.5.d)1°,9,5,Animal husbandry,Mixed animal facility,6.6.a),9.5.d)1° - een intensieve pluimvee- of varkens...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (35057.3 177059.4),https://gpbv.omgeving.vlaanderen.be//api/view/...
1958,1959,BE.VL.000002401.INSTALLATION,Dekervel Bart en Dekervel Aaron,veeteelt,Poperinge,8972,Couthoflaan,13,NaN,9.3.1.d),9,3,Animal husbandry,Poultry,6.6.a),9.3.1.d) - intensieve pluimveehouderij met mee...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (29486 175733),https://gpbv.omgeving.vlaanderen.be//api/view/...


Note: Second extract of data

In [11]:
gdf.to_excel("gpbv_pdf_links.xlsx", index=False)

## Download the PDFs

In [12]:

PDF_DIR = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\PDFs"
os.makedirs(PDF_DIR, exist_ok=True)

def download_pdf(url, doc_id, nummer):
    # Get last 5 characters of the URL and sanitize for filename
    url_suffix = str(url)[-5:].replace('/', '_').replace(':', '_').replace('?', '_').replace('&', '_')
    
    # Create filename: id_nummer_last5chars.pdf
    filename = f"{doc_id}_{nummer}_{url_suffix}.pdf"
    path = os.path.join(PDF_DIR, filename)

    if not os.path.exists(path):
        r = requests.get(url, timeout=30, headers={"User-Agent": "Mozilla/5.0"})
        r.raise_for_status()
        with open(path, "wb") as f:
            f.write(r.content)

    return path

# Read the Pdf information

For this part the most important is to extract the data the most accurate, for avoid the errors and misunderstanding.

In [13]:
test_df = pd.read_excel("gpbv_pdf_links.xlsx")
test_df = test_df.sample(2)

test_df['pdf_path'] = test_df.apply(lambda row: download_pdf(row['pdf_links'], row['id'], row['nummer']), axis=1)
test_df

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,Main,Sub,Category,Sub-Category,rie_rubriek_nummer,vlarem_rubriek_omschrijving,url_fiche,geometry,pdf_links,pdf_path
5537,675,BE.VL.000000824.INSTALLATION,Arkova (Pittem),veeteelt,Pittem,8740,Willekomstraat,2,NaN,9.4.1.d)1°,9,4,Animal husbandry,Other mammals,6.6.b),9.4.1.d)1° - 2.000 plaatsen voor mestvarkens v...,https://gpbv.omgeving.vlaanderen.be/installati...,POINT (74281.375 191737),https://gpbv.omgeving.vlaanderen.be/api/view/v...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...
10418,1544,BE.VL.000001895.INSTALLATION,Veolia Environmental Services - hout,industrie,Gent,9042,Hulsdonk,1,NaN,2.4.3.b)2°,2,4,Waste,Waste management in the context of industrial ...,5.3.b)ii),2.4.3.b)2° - voorbehandeling van afval voor ve...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (109874.5 202882),https://gpbv.omgeving.vlaanderen.be//api/view/...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...


## Extract the information of the PDFs

In this point we extract the information of the PDFs do the traduction to english and stock the data on a txt file for an easier somprehension of the LLM model

In [14]:
# ================== CONFIG ==================

PDF_DIR = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\PDFs"
TXT_DIR = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\TXT"
URL_BASE = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\TXT/"

MAX_WORKERS = 3  # safe for laptop

# ================== HELPERS ==================

def txt_already_exists(pdf_path: str, txt_dir: str) -> bool:

    pdf_filename = os.path.basename(pdf_path)
    base_name = os.path.splitext(pdf_filename)[0]
    txt_path = os.path.join(txt_dir, f"{base_name}.txt")
    return os.path.exists(txt_path)

# ================== MAIN ==================

if __name__ == "__main__":

    # -------- 1. PREPARE ITEMS (WITH RESUME SUPPORT) --------

    items_to_process = []
    already_processed = 0

    for idx, row in test_df.iterrows():
        pdf_path = row["pdf_path"]

        # ⬅️ KEY OPTIMIZATION: skip already processed PDFs
        if txt_already_exists(pdf_path, TXT_DIR):
            already_processed += 1
            continue

        item = row.to_dict()
        item["idx"] = idx
        items_to_process.append(item)

    print("=================================================")
    print(f"Total PDFs in dataframe: {len(test_df)}")
    print(f"Already processed (TXT exists): {already_processed}")
    print(f"PDFs to process now: {len(items_to_process)}")
    print("=================================================\n")

    if not items_to_process:
        print("Nothing to do — all PDFs already processed.")
        exit(0)

    # -------- 2. PROCESS IN PARALLEL --------

    print(f"Starting PDF → TXT extraction with {MAX_WORKERS} workers...\n")

    with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {
            executor.submit(
                process_document_task,
                item,
                PDF_DIR,
                TXT_DIR,
                URL_BASE
            ): item
            for item in items_to_process
        }

        for future in tqdm(
            as_completed(futures),
            total=len(futures),
            desc="Processing PDFs"
        ):
            try:
                idx, result = future.result()

                # Update dataframe only if processing succeeded
                if idx is not None and result:
                    test_df.at[idx, "txt_link"] = result

            except Exception as e:
                # Hard failure in worker (should be rare)
                print(f"Worker failure: {e}")

    print("\n" + "=" * 60)
    print("PDF → TXT processing finished")
    print(f"Total PDFs: {len(test_df)}")
    print(f"Previously processed: {already_processed}")
    print(f"Newly processed this run: {len(items_to_process)}")
    print("=" * 60)

Total PDFs in dataframe: 2
Already processed (TXT exists): 0
PDFs to process now: 2

Starting PDF → TXT extraction with 3 workers...



Processing PDFs:   0%|          | 0/2 [00:00<?, ?it/s]

Processing PDFs: 100%|██████████| 2/2 [01:22<00:00, 41.47s/it]



PDF → TXT processing finished
Total PDFs: 2
Previously processed: 0
Newly processed this run: 2


In [15]:
test_df

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,...,Sub,Category,Sub-Category,rie_rubriek_nummer,vlarem_rubriek_omschrijving,url_fiche,geometry,pdf_links,pdf_path,txt_link
5537,675,BE.VL.000000824.INSTALLATION,Arkova (Pittem),veeteelt,Pittem,8740,Willekomstraat,2,NaN,9.4.1.d)1°,...,4,Animal husbandry,Other mammals,6.6.b),9.4.1.d)1° - 2.000 plaatsen voor mestvarkens v...,https://gpbv.omgeving.vlaanderen.be/installati...,POINT (74281.375 191737),https://gpbv.omgeving.vlaanderen.be/api/view/v...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...
10418,1544,BE.VL.000001895.INSTALLATION,Veolia Environmental Services - hout,industrie,Gent,9042,Hulsdonk,1,NaN,2.4.3.b)2°,...,4,Waste,Waste management in the context of industrial ...,5.3.b)ii),2.4.3.b)2° - voorbehandeling van afval voor ve...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (109874.5 202882),https://gpbv.omgeving.vlaanderen.be//api/view/...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...


# Test if ollama is working ! 

This is the test part for ollama, for better results we can go to the site of ollama and download a more powerfull LLM model for better results if we do the proccess or stock the model in a cloud or a powerfull local machine!

https://ollama.com/library?sort=popular


In [16]:
response = ollama.chat(
    model="llama3.1:8b",
    messages=[{"role": "user", "content": "Hello"}]
)

print(response["message"]["content"])


Is there something I can help you with or would you like to chat?


In [17]:

ollama.chat(
    model="llama3.1:8b",
    messages=[{"role": "user", "content": "test"}]
)


ChatResponse(model='llama3.1:8b', created_at='2026-01-26T08:50:18.6326985Z', done=True, done_reason='stop', total_duration=7408682800, load_duration=165296900, prompt_eval_count=11, prompt_eval_duration=858274700, eval_count=28, eval_duration=6262636300, message=Message(role='assistant', content='It looks like you just tested the system! Is there something else I can help with, or would you like to start a conversation?', thinking=None, images=None, tool_name=None, tool_calls=None), logprobs=None)

# Implemenation of the LLM model analysis

For this proccess the flow is: 

graph TD
    A[ Read TXT File] --> B{ Asset <br/>Keywords Found?}
    
    B -- "No" --> C([End / Skip File])
    B -- "Yes" --> D[LLM Prompting & <br/>Extraction Analysis]
    
    D --> E[Generate Structured <br/>JSON Output]
    E --> F[Flatten JSON <br/>'Explode' to Rows]
    F --> G([Final Normalized <br/>Dataset])

    %% Apply Styles
    class A,D,E,F process;
    class B logic;
    class C,G startEnd;

In [18]:
# ================== CONFIG ==================

MODEL = "llama3.1:8b"

INPUT_TXT_DIR = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\TXT"
OUTPUT_JSON_DIR = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\JSON"

#C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\TXT


In [19]:
client = Client()


def extract_assets_with_llm(text: str):
    prompt = f"""

You are a strict Industrial Energy Auditor. 
Your task: Extract physical assets from the provided TEXT only. 

RULES:\n"

1. DO NOT use outside knowledge.\n
2. DO NOT provide URLs, links, or image paths.\n
3. If no assets are found, return an empty list [].\n
4. if not asset_type found, do not include the entry.\n
4. Return ONLY valid JSON.

Fields:
- asset_type
- capacity_value
- capacity_unit
- count_of_units

TEXT:
{text}
"""
    response = client.chat(
        model=MODEL,
        format="json",
        messages=[{"role": "user", "content": prompt}]
    )
    return json.loads(response["message"]["content"])



In [20]:

os.makedirs(OUTPUT_JSON_DIR, exist_ok=True)

# ================== YOUR ASSET KEYWORDS (SAVED) ==================

ASSET_KEYWORDS = [
    "generator", "genset", "diesel generator", "gas generator",
    "emergency generator", "backup generator",
    "turbine", "gas turbine", "steam turbine",
    "engine", "internal combustion engine", "furnace", 

    "transformer", "power transformer", "distribution transformer",
    "substation", "switchgear", "circuit breaker",
    "electrical panel", "electrical installation",

    "battery", "batteries","battery system", "battery storage",
    "energy storage", "bess",
    "ups", "uninterruptible power supply",

    "kw", "mw", "kva", "mva", "kwh", "mwh", "vah",

    "boiler", "steam boiler", "hot water boiler",
    "heater", "furnace", "kiln", "burner",

    "chiller", "cooling system", "cooling unit",
    "cooling tower",
    "heat pump", "hvac", "air conditioning",

    "hydrogen", "h2",
    "electrolyser", "electrolyzer", "reformer",
    "natural gas", "gas installation",
    "nm3/h", "m3/h", "diesel engine", 

    "compressor", "air compressor",
    "pump", "industrial pump",
    "motor", "electric motor",

    "load shedding", "peak shaving",
    "demand response", "flexibility",
    "energy management system", "ems",

    "capacity", "rated power",
    "nominal power", "maximum power",
]


# ================== FUNCTIONS ==================

def extract_relevant_lines(text: str, max_lines: int = 5000) -> str:
    """
    Finds keyword and grabs current + next line for context.
    """
    lines = text.splitlines()
    relevant_indices = set()

    for i, line in enumerate(lines):
        if any(k in line.lower() for k in ASSET_KEYWORDS):
            relevant_indices.add(i)
            if i + 1 < len(lines):
                relevant_indices.add(i + 1) # Grab the next line too

    sorted_indices = sorted(list(relevant_indices))
    relevant = [f"[L.{idx+1:04}] {lines[idx]}" for idx in sorted_indices]
    return "\n".join(relevant[:max_lines])



def process_file(file_path: str) -> bool:
    filename = os.path.basename(file_path)
    
    try:
        with open(file_path, encoding="utf-8", errors="ignore") as f:
            text = f.read()
            
        # Quick Filter
        if not any(k in text.lower() for k in ASSET_KEYWORDS):
            return False

        relevant_text = extract_relevant_lines(text)
        if not relevant_text.strip():
            return False

        assets = extract_assets_with_llm(relevant_text)

        output_path = os.path.join(OUTPUT_JSON_DIR, filename.replace(".txt", ".json"))
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump({"source": filename, "assets": assets}, f, indent=2)

        return True
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return False

# ================== MAIN ==================

if __name__ == "__main__":
    # 1. Gather all files
    all_files = [f for f in os.listdir(INPUT_TXT_DIR) if f.endswith(".txt")]
    
    # 2. Check status
    to_process = []
    already_done_count = 0
    
    for f in all_files:
        json_filename = f.replace(".txt", ".json")
        if os.path.exists(os.path.join(OUTPUT_JSON_DIR, json_filename)):
            already_done_count += 1
        else:
            to_process.append(f)

    print("\n" + "=" * 50)
    print(f"REPORT")
    print(f"Total Files in folder:    {len(all_files)}")
    print(f"Already Processed:        {already_done_count}")
    print(f"New Files to process:     {len(to_process)}")
    print("=" * 50 + "\n")

    if not to_process:
        print("No new files found. Everything is up to date!")
    else:
        # 3. Process remaining
        newly_processed = 0
        skipped = 0

        for f in tqdm(to_process, desc="Processing new documents"):
            ok = process_file(os.path.join(INPUT_TXT_DIR, f))
            if ok:
                newly_processed += 1
            else:
                skipped += 1

        print(f"\nProcessing Complete!")
        print(f"Newly Created: {newly_processed}")
        print(f"Skipped (no assets): {skipped}")


REPORT
Total Files in folder:    10
Already Processed:        2
New Files to process:     8



Processing new documents:   0%|          | 0/8 [00:00<?, ?it/s]

Processing new documents:  12%|█▎        | 1/8 [02:51<20:01, 171.69s/it]

: 

: 

In [1]:
test_df

NameError: name 'test_df' is not defined

# .JSON extract data for the assets and analysis

In [ ]:
json_folder = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\JSON"

In [ ]:
def extract_json_data(txt_path):
    # 1. Get the filename without extension (e.g., 1117_BE...f5f1a)
    file_base = os.path.splitext(os.path.basename(txt_path))[0]
    
    # 2. Construct the path to the JSON folder
    # Adjust 'JSON_FOLDER_PATH' to your actual JSON directory
    
    json_path = os.path.join(json_folder, file_base + ".json")
    
    # 3. Read the JSON file
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
            # Return the list of assets
            return data.get("assets", {}).get("assets", [])
    return []

# Apply the function to create a new column with the list of assets
test_df['assets_list'] = test_df['txt_link'].apply(extract_json_data)

# 4. "Explode" the list into multiple rows
df_expanded = test_df.explode('assets_list')

# 5. Expand the dictionary into individual columns
# This turns {'asset_type': 'X', ...} into separate columns
assets_df = df_expanded['assets_list'].apply(pd.Series)

# Combine with the original dataframe (dropping the helper column)
final_df = pd.concat([df_expanded.drop('assets_list', axis=1), assets_df], axis=1)

# Clean up: Replace NaN or empty strings if necessary
final_df = final_df.fillna("")

In [ ]:
final_df

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,...,url_fiche,geometry,pdf_links,pdf_path,txt_link,asset_type,capacity_value,capacity_unit,count_of_units,0
11938,1850,BE.VL.000002273.INSTALLATION,Greenergy,industrie,Herselt,2230,Dieperstraat,110,,2.4.3.b)1°,...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (182947.9 193675.6),https://gpbv.omgeving.vlaanderen.be//api/view/...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,engine,930 kW,kW,1,
11938,1850,BE.VL.000002273.INSTALLATION,Greenergy,industrie,Herselt,2230,Dieperstraat,110,,2.4.3.b)1°,...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (182947.9 193675.6),https://gpbv.omgeving.vlaanderen.be//api/view/...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,engine,1190 kW,kW,2,
11938,1850,BE.VL.000002273.INSTALLATION,Greenergy,industrie,Herselt,2230,Dieperstraat,110,,2.4.3.b)1°,...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (182947.9 193675.6),https://gpbv.omgeving.vlaanderen.be//api/view/...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,engine,366 kW,kW,1,
11938,1850,BE.VL.000002273.INSTALLATION,Greenergy,industrie,Herselt,2230,Dieperstraat,110,,2.4.3.b)1°,...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (182947.9 193675.6),https://gpbv.omgeving.vlaanderen.be//api/view/...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,dual fuel engine,2000 kW,kW,2,
11938,1850,BE.VL.000002273.INSTALLATION,Greenergy,industrie,Herselt,2230,Dieperstraat,110,,2.4.3.b)1°,...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (182947.9 193675.6),https://gpbv.omgeving.vlaanderen.be//api/view/...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,engine,1222 kW,kW,3,
11938,1850,BE.VL.000002273.INSTALLATION,Greenergy,industrie,Herselt,2230,Dieperstraat,110,,2.4.3.b)1°,...,https://gpbv.omgeving.vlaanderen.be//installat...,POINT (182947.9 193675.6),https://gpbv.omgeving.vlaanderen.be//api/view/...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,emergency power generator,125 kW,kW,1,
8227,1146,BE.VL.000001389.INSTALLATION,Ganfraco/Snaet Geert,veeteelt,Lichtervelde,8810,Slunsjestraat,2,,9.4.1.d)1°,...,https://gpbv.omgeving.vlaanderen.be/installati...,POINT (65428.75 189509.5),https://gpbv.omgeving.vlaanderen.be/api/view/v...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,,,,,


# Test of .py funtions for production enviroment

In [ ]:
import llm_utils

In [2]:

ASSET_KEYWORDS = [
    "furnace","generator", "genset", "diesel generator", "gas generator",
    "emergency generator", "backup generator",
    "turbine", "gas turbine", "steam turbine",
    "engine", "internal combustion engine", "furnace", 

    "transformer", "power transformer", "distribution transformer",
    "substation", "switchgear", "circuit breaker",
    "electrical panel", "electrical installation",

    "battery", "batteries","battery system", "battery storage",
    "energy storage", "bess",
    "ups", "uninterruptible power supply",

    "kw", "mw", "kva", "mva", "kwh", "mwh", "vah",

    "boiler", "steam boiler", "hot water boiler",
    "heater", "furnace", "kiln", "burner",

    "chiller", "cooling system", "cooling unit",
    "cooling tower",
    "heat pump", "hvac", "air conditioning",

    "hydrogen", "h2",
    "electrolyser", "electrolyzer", "reformer",
    "natural gas", "gas installation",
    "nm3/h", "m3/h", "diesel engine", 

    "compressor", "air compressor",
    "pump", "industrial pump",
    "motor", "electric motor",

    "load shedding", "peak shaving",
    "demand response", "flexibility",
    "energy management system", "ems",

    "capacity", "rated power",
    "nominal power", "maximum power",
]

In [3]:
import re
import json
from ollama import Client
import config

client = Client()

# ======================================================
# 1. SMART KEYWORD EXTRACTION (NO REGEX NOISE)
# ======================================================
def extract_relevant_lines(text, max_lines=5000):
    """
    Extract ONLY lines with real asset keywords.
    NO regex pre-extraction - let LLM decide what's an asset.
    """
    print("\n" + "="*70)
    print("📋 FUNCTION: extract_relevant_lines()")
    print("="*70)
    
    lines = text.splitlines()
    print(f"📊 Total lines in text: {len(lines)}")
    print(f"📊 Text length: {len(text)} characters")
    
    relevant_indices = set()
    
    EXCLUDE_KEYWORDS = [
        "hydrogen fluoride", "sodium hydroxide", "emission", "limit",
        "concentration", "mg/nm3", "regulation", "decree", "permit",
        "compliance", "monitoring", "sampling", "standard", "requirement"
    ]
    
    print(f"🚫 Exclude keywords: {len(EXCLUDE_KEYWORDS)} patterns")
    print(f"🔍 Asset keywords to search: {len(config.ASSET_KEYWORDS)} patterns\n")

    excluded_count = 0
    matched_count = 0
    keyword_matches = {}

    for i, line in enumerate(lines):
        line_lower = line.lower()
        
        # SKIP regulatory noise
        if any(excl in line_lower for excl in EXCLUDE_KEYWORDS):
            excluded_count += 1
            continue
        
        # ONLY include if asset keyword present
        for keyword in config.ASSET_KEYWORDS:
            if keyword in line_lower:
                # Grab context: this line + next 2 lines
                for j in range(i, min(i + 3, len(lines))):
                    relevant_indices.add(j)
                
                matched_count += 1
                if keyword not in keyword_matches:
                    keyword_matches[keyword] = 0
                keyword_matches[keyword] += 1
                break  # Don't count same line twice

    print(f"⏭️  Lines excluded (regulatory/noise): {excluded_count}")
    print(f"✅ Lines matched with asset keywords: {matched_count}")
    print(f"📦 Unique context lines extracted: {len(relevant_indices)}\n")
    
    # Show top keywords found
    if keyword_matches:
        print("🏆 Top 10 keywords found:")
        for kw, count in sorted(keyword_matches.items(), key=lambda x: x[1], reverse=True)[:10]:
            print(f"   • {kw:30s} → {count:3d} occurrences")
        print()

    sorted_indices = sorted(relevant_indices)
    relevant = [lines[idx] for idx in sorted_indices]
    result = "\n".join(relevant[:max_lines])
    
    print(f"📄 Final extracted text length: {len(result)} characters")
    print(f"📄 Lines in output: {len(relevant)}")
    print(f"✂️  Capped at max_lines: {max_lines}")
    print("="*70 + "\n")
    
    return result

# ======================================================
# 2. DIRECT LLM EXTRACTION (NO REGEX PREPROCESSING)
# ======================================================
def extract_assets_from_text(text, debug=False):
    """
    Extract assets directly with the proven prompt.
    NO regex pre-extraction step.
    """
    print("\n" + "="*70)
    print("🤖 FUNCTION: extract_assets_from_text()")
    print("="*70)
    print(f"⏱️  Timestamp: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🔧 Debug mode: {debug}\n")
    
    # Step 1: Filter relevant lines
    print("STEP 1️⃣: Extracting relevant lines...")
    relevant = extract_relevant_lines(text)
    
    if debug:
        print(f"📝 Relevant text extracted: {len(relevant)} chars")
    
    if not relevant.strip():
        print("⚠️  No relevant lines found - returning empty assets")
        print("="*70 + "\n")
        return {"assets": []}

    print("STEP 2️⃣: Preparing LLM prompt...")
    
    # Step 2: Direct LLM call with YOUR PROVEN PROMPT
    prompt = f"""You are a strict Industrial Energy Auditor. 
Your task: Extract physical assets from the provided TEXT only. 

RULES:

1. DO NOT use outside knowledge.
2. DO NOT provide URLs, links, or image paths.
3. If no assets are found, return an empty list [].
4. If not asset_type found, do not include the entry.
5. Return ONLY valid JSON.

Fields:
- asset_type
- capacity_value
- capacity_unit
- count_of_units

TEXT:
{relevant}
"""
    
    print(f"📋 Prompt length: {len(prompt)} characters")
    print(f"📋 Prompt preview (first 200 chars):\n{prompt[:200]}...\n")

    try:
        print("STEP 3️⃣: Sending request to Ollama LLM...")
        print(f"🤖 Model: {config.OLLAMA_MODEL}")
        print(f"📤 Input text length: {len(relevant)} chars")
        
        import time
        start_time = time.time()
        
        response = client.chat(
            model=config.OLLAMA_MODEL,
            format="json",
            messages=[{"role": "user", "content": prompt}]
        )
        
        elapsed_time = time.time() - start_time
        print(f"⏱️  LLM processing time: {elapsed_time:.2f} seconds")
        
        raw_text = response["message"]["content"].strip()
        
        print(f"\nSTEP 4️⃣: Processing LLM response...")
        print(f"📊 Response length: {len(raw_text)} characters")
        print(f"📊 Response preview (first 300 chars):\n{raw_text[:300]}\n")
        
        # Handle empty response
        if not raw_text or raw_text == "{}":
            print("⚠️  LLM returned empty response: '{}'")
            print("="*70 + "\n")
            return {"assets": []}
        
        # Parse JSON
        print("🔄 Parsing JSON response...")
        data = json.loads(raw_text)
        print(f"✅ JSON parsed successfully")
        print(f"📊 Root type: {type(data).__name__}")
        
        # Normalize response format
        print("\nNormalizing response format...")
        if isinstance(data, dict):
            print(f"   - Response is dict with keys: {list(data.keys())}")
            if "assets" in data:
                assets = data.get("assets", [])
                print(f"   - Found 'assets' key with {len(assets)} items")
            else:
                # Single asset object - wrap it
                if "asset_type" in data:
                    assets = [data]
                    print(f"   - Single asset detected, wrapping in list")
                else:
                    assets = []
                    print(f"   - No 'asset_type' found, returning empty")
        elif isinstance(data, list):
            assets = data
            print(f"   - Response is list with {len(assets)} items")
        else:
            assets = []
            print(f"   - Response is {type(data).__name__}, cannot process")
        
        # Validate: only keep items with asset_type
        print("\nValidating assets...")
        print(f"📊 Total items to validate: {len(assets)}")
        
        invalid_count = 0
        for idx, a in enumerate(assets):
            if not isinstance(a, dict):
                print(f"   ⚠️  Item {idx}: Not a dict (type: {type(a).__name__})")
                invalid_count += 1
            elif not a.get("asset_type"):
                print(f"   ⚠️  Item {idx}: Missing 'asset_type'")
                invalid_count += 1
        
        validated = [
            a for a in assets 
            if isinstance(a, dict) and a.get("asset_type")
        ]
        
        print(f"📊 Invalid items removed: {invalid_count}")
        print(f"✅ Valid assets extracted: {len(validated)}")
        
        if validated:
            print("\n🎯 Extracted Assets:")
            for idx, asset in enumerate(validated, 1):
                asset_type = asset.get("asset_type", "N/A")
                capacity = asset.get("capacity_value", "N/A")
                unit = asset.get("capacity_unit", "N/A")
                count = asset.get("count_of_units", "N/A")
                print(f"   {idx}. {asset_type:30s} | {capacity:10s} {unit:10s} | Count: {count}")
        
        print("\n" + "="*70 + "\n")
        return {"assets": validated}
        
    except json.JSONDecodeError as e:
        print(f"\n❌ JSON PARSING ERROR")
        print(f"   Error: {e}")
        print(f"   Raw response (first 500 chars):\n{raw_text[:500]}")
        print("="*70 + "\n")
        return {"assets": []}
    except Exception as e:
        print(f"\n❌ UNEXPECTED ERROR")
        print(f"   Error type: {type(e).__name__}")
        print(f"   Error message: {e}")
        import traceback
        print(f"   Traceback:\n{traceback.format_exc()}")
        print("="*70 + "\n")
        return {"assets": []}


# ======================================================
# 3. DEBUG FUNCTION (VERBOSE OUTPUT)
# ======================================================
def debug_extract_relevant_lines(text):
    """DEBUG version with extra logging"""
    print("\n" + "="*70)
    print("🐛 DEBUG FUNCTION: debug_extract_relevant_lines()")
    print("="*70 + "\n")
    
    lines = text.splitlines()
    relevant_indices = set()
    
    EXCLUDE_KEYWORDS = [
        "hydrogen fluoride", "sodium hydroxide", "emission", "limit", 
        "concentration", "mg/nm3", "regulation", "decree", "permit",
        "compliance", "monitoring", "sampling", "standard", "requirement"
    ]
    
    print(f"📊 Input Statistics:")
    print(f"   - Total lines: {len(lines)}")
    print(f"   - Total characters: {len(text)}")
    print(f"   - Average line length: {len(text)//len(lines) if lines else 0} chars")
    print(f"   - Exclude patterns: {len(EXCLUDE_KEYWORDS)}")
    print(f"   - Asset keywords: {len(config.ASSET_KEYWORDS)}\n")
    
    excluded_count = 0
    keyword_matches = {}
    
    for i, line in enumerate(lines):
        line_lower = line.lower()
        
        # Check if excluded
        is_excluded = False
        for excl in EXCLUDE_KEYWORDS:
            if excl in line_lower:
                excluded_count += 1
                is_excluded = True
                break
        
        if is_excluded:
            continue
        
        # Check if included
        for keyword in config.ASSET_KEYWORDS:
            if keyword in line_lower:
                relevant_indices.add(i)
                
                if keyword not in keyword_matches:
                    keyword_matches[keyword] = []
                keyword_matches[keyword].append((i+1, line[:70]))
                
                # Grab next 2 lines for context
                if i + 1 < len(lines):
                    relevant_indices.add(i + 1)
                if i + 2 < len(lines):
                    relevant_indices.add(i + 2)
                break
    
    sorted_indices = sorted(list(relevant_indices))
    
    print(f"📊 Processing Results:")
    print(f"   - Lines excluded (regulatory): {excluded_count}")
    print(f"   - Lines matched with keywords: {len(keyword_matches)}")
    print(f"   - Total relevant lines (with context): {len(sorted_indices)}\n")
    
    if keyword_matches:
        print(f"🔍 Keywords Found (top 15):")
        for kw, occurrences in sorted(keyword_matches.items(), key=lambda x: len(x[1]), reverse=True)[:15]:
            print(f"   • {kw:35s} → {len(occurrences):3d} occurrences")
            for line_num, preview in occurrences[:2]:  # Show first 2 occurrences
                print(f"      [L.{line_num:04d}] {preview}...")
        print()
    
    relevant = [f"[L.{idx+1:04d}] {lines[idx]}" for idx in sorted_indices]
    result = "\n".join(relevant[:5000])
    
    print(f"📄 Output Statistics:")
    print(f"   - Output length: {len(result)} characters")
    print(f"   - Output lines: {len(relevant)}")
    print(f"   - Compression ratio: {len(result)/len(text)*100:.1f}% of original")
    print("\n" + "="*70 + "\n")
    
    return result

In [4]:
ASSET_KEYWORDS

['furnace',
 'generator',
 'genset',
 'diesel generator',
 'gas generator',
 'emergency generator',
 'backup generator',
 'turbine',
 'gas turbine',
 'steam turbine',
 'engine',
 'internal combustion engine',
 'furnace',
 'transformer',
 'power transformer',
 'distribution transformer',
 'substation',
 'switchgear',
 'circuit breaker',
 'electrical panel',
 'electrical installation',
 'battery',
 'batteries',
 'battery system',
 'battery storage',
 'energy storage',
 'bess',
 'ups',
 'uninterruptible power supply',
 'kw',
 'mw',
 'kva',
 'mva',
 'kwh',
 'mwh',
 'vah',
 'boiler',
 'steam boiler',
 'hot water boiler',
 'heater',
 'furnace',
 'kiln',
 'burner',
 'chiller',
 'cooling system',
 'cooling unit',
 'cooling tower',
 'heat pump',
 'hvac',
 'air conditioning',
 'hydrogen',
 'h2',
 'electrolyser',
 'electrolyzer',
 'reformer',
 'natural gas',
 'gas installation',
 'nm3/h',
 'm3/h',
 'diesel engine',
 'compressor',
 'air compressor',
 'pump',
 'industrial pump',
 'motor',
 'electri

In [5]:
import llm_utils
import json

print("🚀 STARTING ASSET EXTRACTION TEST\n")

with open(
    r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\TXT\1_BE.VL.000000002.INSTALLATION_5a5cb.txt", 
    "r", encoding="utf-8", errors="ignore"
) as f:
    text = f.read()

print(f"📂 File loaded: {len(text)} characters\n")

# Run extraction with full debug output
result = extract_assets_from_text(text, debug=True)

print("\n" + "="*70)
print("📋 FINAL RESULT")
print("="*70)
print(json.dumps(result, indent=2))



# This is the best model for the instant!

🚀 STARTING ASSET EXTRACTION TEST

📂 File loaded: 59698 characters


🤖 FUNCTION: extract_assets_from_text()
⏱️  Timestamp: 2026-01-26 14:25:49
🔧 Debug mode: True

STEP 1️⃣: Extracting relevant lines...

📋 FUNCTION: extract_relevant_lines()
📊 Total lines in text: 1107
📊 Text length: 59698 characters
🚫 Exclude keywords: 14 patterns
🔍 Asset keywords to search: 104 patterns

⏭️  Lines excluded (regulatory/noise): 122
✅ Lines matched with asset keywords: 201
📦 Unique context lines extracted: 355

🏆 Top 10 keywords found:
   • kw                             →  42 occurrences
   • container                      →  34 occurrences
   • fuel oil                       →  20 occurrences
   • hydrogen                       →  19 occurrences
   • pump                           →  13 occurrences
   • well                           →  12 occurrences
   • groundwater                    →  12 occurrences
   • furnace                        →   8 occurrences
   • m3                             →   6 occur

In [6]:
import re
import json
from ollama import Client
import config
import time

client = Client()

# ======================================================
# 1. AGGRESSIVE KEYWORD EXTRACTION (REDUCE NOISE)
# ======================================================
def extract_relevant_lines(text, max_lines=2000):
    """
    Extract ONLY lines with real asset keywords.
    AGGRESSIVE filtering to reduce hallucinations.
    """
    print("\n" + "="*70)
    print("📋 FUNCTION: extract_relevant_lines()")
    print("="*70)
    
    lines = text.splitlines()
    print(f"📊 Total lines in text: {len(lines)}")
    print(f"📊 Text length: {len(text)} characters")
    
    relevant_indices = set()
    
    # MORE aggressive exclusions
    EXCLUDE_KEYWORDS = [
        "hydrogen fluoride", "sodium hydroxide", "emission", "limit",
        "concentration", "mg/nm3", "regulation", "decree", "permit",
        "compliance", "monitoring", "sampling", "standard", "requirement",
        "analysis", "review", "contested", "decision", "document", "expansion",
        "approval", "permission", "authorization", "notice", "announcement"
    ]
    
    print(f"🚫 Exclude keywords: {len(EXCLUDE_KEYWORDS)} patterns")
    print(f"🔍 Asset keywords to search: {len(config.ASSET_KEYWORDS)} patterns\n")

    excluded_count = 0
    matched_count = 0
    keyword_matches = {}

    for i, line in enumerate(lines):
        line_lower = line.lower()
        
        # SKIP regulatory/analysis noise
        if any(excl in line_lower for excl in EXCLUDE_KEYWORDS):
            excluded_count += 1
            continue
        
        # ONLY include if asset keyword present
        for keyword in config.ASSET_KEYWORDS:
            if keyword in line_lower:
                # Grab context: this line + next 1 line only (not 3!)
                for j in range(i, min(i + 2, len(lines))):
                    relevant_indices.add(j)
                
                matched_count += 1
                if keyword not in keyword_matches:
                    keyword_matches[keyword] = 0
                keyword_matches[keyword] += 1
                break

    print(f"⏭️  Lines excluded (regulatory/noise): {excluded_count}")
    print(f"✅ Lines matched with asset keywords: {matched_count}")
    print(f"📦 Unique context lines extracted: {len(relevant_indices)}\n")
    
    if keyword_matches:
        print("🏆 Top 10 keywords found:")
        for kw, count in sorted(keyword_matches.items(), key=lambda x: x[1], reverse=True)[:10]:
            print(f"   • {kw:30s} → {count:3d} occurrences")
        print()

    sorted_indices = sorted(relevant_indices)
    relevant = [lines[idx] for idx in sorted_indices]
    result = "\n".join(relevant[:max_lines])
    
    print(f"📄 Final extracted text length: {len(result)} characters")
    print(f"📄 Lines in output: {len(relevant)}")
    print(f"✂️  Capped at max_lines: {max_lines}")
    print("="*70 + "\n")
    
    return result

# ======================================================
# 2. IMPROVED LLM EXTRACTION WITH SAFETY
# ======================================================
def extract_assets_from_text(text, debug=False):
    """
    Extract assets with safety measures.
    """
    print("\n" + "="*70)
    print("🤖 FUNCTION: extract_assets_from_text()")
    print("="*70)
    print(f"⏱️  Timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🔧 Debug mode: {debug}\n")
    
    # Step 1: Filter relevant lines
    print("STEP 1️⃣: Extracting relevant lines...")
    relevant = extract_relevant_lines(text)
    
    if debug:
        print(f"📝 Relevant text extracted: {len(relevant)} chars")
    
    if not relevant.strip():
        print("⚠️  No relevant lines found - returning empty assets")
        print("="*70 + "\n")
        return {"assets": []}

    print("STEP 2️⃣: Preparing LLM prompt...")
    
    # SIMPLIFIED PROMPT - More direct
    prompt = f"""Extract assets from TEXT. Return ONLY JSON array.

RULES:
- Extract ONLY from TEXT, NO outside knowledge
- Return valid JSON array ONLY
- Each item must have asset_type
- Return [] if no assets found

Format: [{{"asset_type": "X", "capacity_value": "Y", "capacity_unit": "Z", "count_of_units": "N"}}]

TEXT:
{relevant}
"""
    
    print(f"📋 Prompt length: {len(prompt)} characters")
    print(f"📋 First 150 chars: {prompt[:150]}...\n")

    try:
        print("STEP 3️⃣: Sending request to Ollama LLM...")
        print(f"🤖 Model: {config.OLLAMA_MODEL}")
        print(f"📤 Input text length: {len(relevant)} chars")
        
        start_time = time.time()
        
        response = client.chat(
            model=config.OLLAMA_MODEL,
            format="json",
            messages=[{"role": "user", "content": prompt}],
            stream=False
        )
        
        elapsed_time = time.time() - start_time
        print(f"⏱️  LLM processing time: {elapsed_time:.2f} seconds")
        
        raw_text = response["message"]["content"].strip()
        
        print(f"\nSTEP 4️⃣: Processing LLM response...")
        print(f"📊 Response length: {len(raw_text)} characters")
        print(f"📊 Response (full):\n{raw_text}\n")
        
        # Handle empty response
        if not raw_text or raw_text == "{}":
            print("⚠️  LLM returned empty response")
            print("="*70 + "\n")
            return {"assets": []}
        
        # Parse JSON
        print("🔄 Parsing JSON response...")
        try:
            data = json.loads(raw_text)
        except json.JSONDecodeError as e:
            print(f"❌ Invalid JSON: {e}")
            print(f"Attempting to extract JSON array from response...")
            # Try to extract JSON array manually
            match = re.search(r'\[.*\]', raw_text, re.DOTALL)
            if match:
                try:
                    data = json.loads(match.group(0))
                    print("✅ Extracted JSON array from response")
                except:
                    print("❌ Could not extract valid JSON")
                    print("="*70 + "\n")
                    return {"assets": []}
            else:
                print("="*70 + "\n")
                return {"assets": []}
        
        print(f"✅ JSON parsed successfully")
        print(f"📊 Root type: {type(data).__name__}")
        
        # Normalize response format
        print("\nNormalizing response format...")
        if isinstance(data, list):
            assets = data
            print(f"   - Response is list with {len(assets)} items")
        elif isinstance(data, dict):
            if "assets" in data:
                assets = data.get("assets", [])
                print(f"   - Found 'assets' key with {len(assets)} items")
            elif "asset_type" in data:
                assets = [data]
                print(f"   - Single asset detected, wrapping in list")
            else:
                assets = []
                print(f"   - No assets found in dict")
        else:
            assets = []
            print(f"   - Response is {type(data).__name__}, cannot process")
        
        # Validate: only keep items with asset_type
        print("\nValidating assets...")
        print(f"📊 Total items to validate: {len(assets)}")
        
        invalid_count = 0
        for idx, a in enumerate(assets):
            if not isinstance(a, dict):
                print(f"   ⚠️  Item {idx}: Not a dict")
                invalid_count += 1
            elif not a.get("asset_type"):
                print(f"   ⚠️  Item {idx}: Missing 'asset_type'")
                invalid_count += 1
        
        validated = [
            a for a in assets 
            if isinstance(a, dict) and a.get("asset_type")
        ]
        
        print(f"📊 Invalid items removed: {invalid_count}")
        print(f"✅ Valid assets extracted: {len(validated)}")
        
        if validated:
            print("\n🎯 Extracted Assets:")
            for idx, asset in enumerate(validated, 1):
                asset_type = asset.get("asset_type", "N/A")
                capacity = asset.get("capacity_value", "N/A")
                unit = asset.get("capacity_unit", "N/A")
                count = asset.get("count_of_units", "N/A")
                print(f"   {idx}. {asset_type:30s} | {capacity:10s} {unit:10s} | Count: {count}")
        
        print("\n" + "="*70 + "\n")
        return {"assets": validated}
        
    except Exception as e:
        print(f"\n❌ UNEXPECTED ERROR")
        print(f"   Error type: {type(e).__name__}")
        print(f"   Error message: {e}")
        import traceback
        print(f"   Traceback:\n{traceback.format_exc()}")
        print("="*70 + "\n")
        return {"assets": []}


# ======================================================
# 3. DEBUG FUNCTION
# ======================================================
def debug_extract_relevant_lines(text):
    """DEBUG version with extra logging"""
    print("\n" + "="*70)
    print("🐛 DEBUG FUNCTION: debug_extract_relevant_lines()")
    print("="*70 + "\n")
    
    lines = text.splitlines()
    relevant_indices = set()
    
    EXCLUDE_KEYWORDS = [
        "hydrogen fluoride", "sodium hydroxide", "emission", "limit", 
        "concentration", "mg/nm3", "regulation", "decree", "permit",
        "compliance", "monitoring", "sampling", "standard", "requirement",
        "analysis", "review", "contested", "decision", "document", "expansion"
    ]
    
    print(f"📊 Input Statistics:")
    print(f"   - Total lines: {len(lines)}")
    print(f"   - Total characters: {len(text)}")
    print(f"   - Exclude patterns: {len(EXCLUDE_KEYWORDS)}")
    print(f"   - Asset keywords: {len(config.ASSET_KEYWORDS)}\n")
    
    excluded_count = 0
    keyword_matches = {}
    
    for i, line in enumerate(lines):
        line_lower = line.lower()
        
        if any(excl in line_lower for excl in EXCLUDE_KEYWORDS):
            excluded_count += 1
            continue
        
        for keyword in config.ASSET_KEYWORDS:
            if keyword in line_lower:
                relevant_indices.add(i)
                if keyword not in keyword_matches:
                    keyword_matches[keyword] = []
                keyword_matches[keyword].append((i+1, line[:70]))
                if i + 1 < len(lines):
                    relevant_indices.add(i + 1)
                break
    
    sorted_indices = sorted(list(relevant_indices))
    
    print(f"📊 Processing Results:")
    print(f"   - Lines excluded: {excluded_count}")
    print(f"   - Keywords matched: {len(keyword_matches)}")
    print(f"   - Total relevant lines: {len(sorted_indices)}\n")
    
    if keyword_matches:
        print(f"🔍 Keywords Found (top 15):")
        for kw, occurrences in sorted(keyword_matches.items(), key=lambda x: len(x[1]), reverse=True)[:15]:
            print(f"   • {kw:35s} → {len(occurrences):3d} occurrences")
    
    relevant = [f"[L.{idx+1:04d}] {lines[idx]}" for idx in sorted_indices]
    result = "\n".join(relevant[:2000])
    
    print(f"\n📄 Output Statistics:")
    print(f"   - Output length: {len(result)} characters")
    print(f"   - Output lines: {len(relevant)}")
    print(f"   - Compression ratio: {len(result)/len(text)*100:.1f}%")
    print("\n" + "="*70 + "\n")
    
    return result

In [ ]:
import json

with open(r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Data_stock\TXT\1_BE.VL.000000002.INSTALLATION_5a5cb.txt", "r", encoding="utf-8", errors="ignore") as f:
    text = f.read()

result = extract_assets_from_text(text, debug=True)
print(json.dumps(result, indent=2))


🤖 FUNCTION: extract_assets_from_text()
⏱️  Timestamp: 2026-01-26 14:51:57
🔧 Debug mode: True

STEP 1️⃣: Extracting relevant lines...

📋 FUNCTION: extract_relevant_lines()
📊 Total lines in text: 1107
📊 Text length: 59698 characters
🚫 Exclude keywords: 25 patterns
🔍 Asset keywords to search: 104 patterns

⏭️  Lines excluded (regulatory/noise): 165
✅ Lines matched with asset keywords: 198
📦 Unique context lines extracted: 291

🏆 Top 10 keywords found:
   • kw                             →  42 occurrences
   • container                      →  34 occurrences
   • fuel oil                       →  20 occurrences
   • hydrogen                       →  19 occurrences
   • well                           →  12 occurrences
   • groundwater                    →  12 occurrences
   • pump                           →  11 occurrences
   • furnace                        →   8 occurrences
   • m3                             →   6 occurrences
   • litres                         →   6 occurrences

📄 Fin